# Kalman Filter Function in PyGom

The Kalman filter method assumes that two set of inputs are available - outputs from a model,and observations of at least some of these outputs. It is further assumed that both these outputs have some noise associated with them, so that neither the model outputs nor the observations are entirely reliable.The Kalman filter technique takes these two inputs and returns an output which is the most likely value of all the model variables, taking into account the uncertainty associated with both inputs.

# Methodology

As described earlier the Kalmam filter method allows an estimate to be made of the most most likely variable values given both model outputs and observations. 
So, if model results are available time step by time step, and observations are available at these same time steps, then this suggests a way of applying the Kalman filter (KF) method to improve a time step by time step numerical model:
1. Starting from model outputs at time t  -Xt say - apply the KF to get filtered results - XFt say -at time t.
2. Use these filtered results as the starting point of another applicaton of the model over the next time step ,to get model results Xt+1
3. Repeat step 1., now at time t+1,to get filtered results at time t+1,XFt+1 ,and so on.

The original motivation behind putting the KF method into PyGom was to allow a time varying beta to be modelled in a compartmental model. In this approach, at step 2. above beta is treated as a variable, which the KF can therefore modify, whereas in steps 1 and 3 beta is regarded as a fixed parameter. So,the KF feature has been implemented with this application in mind.

Full details are given in [1].


# Implementation Within PyGom
It is anticipated that the Kalamn filter application in PyGom will mostly be applied using the def IntegrateWithKalmanFilter method for deterministic models.This is equivalent to the integrate method for determinisitc models, but in this case after every internal time step within the integration process the Kalman filter is used update the latest output of the model to take of account of observed results. In this way the code applies the methodolgy described in the earlier section

The relevant lines of code are shown below:

  for i in range(len(observations)-1):
      
      # predict step         
             xnew =self.integrate(t[i+1])
            
     # update step
             xfilter,Pnew,KG = self.Kalman_Filter(observations[i],observed_states,model,xnew[-1],P,t[i+1],t[i+1]-t[i],sigmaw2,sigmar2,beta_state,Bounds) 
             #xfilter = xnew[-1]
           
             model_outputs.append(xfilter)
             P = Pnew
             self.initial_values = (xfilter,t[i+1])
             
        return model_outputs
        
        
So matching the methodology with the code:
  xnew =self.integrate(t[i+1]) performs step 1.
  
  xfilter,Pnew,KG = self.Kalman_Filter  perform step 2
  
  self.initial_values = (xfilter,t[i+1]) then sets the starting point of the next model application to the 'filter' resluts returne by the previous application of the KF


In [ ]:
The steps for setting up are as follows

In [2]:
#Import the PyGom package:
from pygom import Transition, ODEVariable, DeterministicOde

ImportError: cannot import name 'Transition' from 'pygom' (unknown location)

In [ ]:
The IntegrateWithKalamnFilter function has input arguments as shown below:
     def IntegrateWithKalmanFilter( self,model, x0,t, observations, observed_states, sigmap2,sigmaw2,sigmar2,full_output=False,beta_state=None,Bounds =None):
        # model is assumed to be a determinisitc ode model
        # t is a list of times for which results are required
        # observations is th lsit of observations at these times
        # observed_states is a list of the indices of the observed states 
        # So, observations should be a list with the same length as t . Each entry in observations
        # should be a list of length observed_states

The input arguments are:
    model:
    This is a PyGom determinisitc ODE model
    X0:
    The starting value  of the variables
    t:
    A list of the time intervals over which the integration is performed
    observations: 
    A list of observations of at least one of the varaibles which feature in the model(not all of the model variables need to be observed)
    observed_states:
    A list of those model states which are being observed (see above)
    sigmap2,sigmaw2,sigmar2 - the Kalman filter process involves three covariance matrices - the estimate covariance matrix(for which an a prior estimate must be provided at the start) , the process noise, and the observation noise . These matrices are assumed to be diagonal, with  
    full_output - this is Standard PyGom input
    beta_state, Bounds - these are alternative ways of specifying bounds on the  variable outputs . Two methods are possible. beta_state allows the user to specify a particular varaible, and that variable is then not allowed to be negative. Alternatively,  the Bounds input can be used to specify an upper and lower bound for each variabl.For a particualr variable either bound, or both bounds, might be "None", meaning that no bounds are applied for that Variable. 

In [ ]:
#The next cells demonstrate how set up and run a Kalman Filter exmaple .In this case 

In [ ]:
#Basic setting up of model without Kalman Filter. So, in this formulation beta is a parameter, not a variable
import scipy.stats
state_list = ['S', 'I', 'R']
param_list = ['beta', 'gamma','N']
transition_list = [
                  Transition(origin='S', destination='I', equation='beta*S*I/N', transition_type="T"),
                  Transition(origin='I', destination='R', equation='gamma*I', transition_type="T")
                 ]

d = dict()
d['beta']  = 1.8
d['gamma'] = 0.2
d['N']     = 1000

t1 = 100
N = 1000

x0=  [ 900.0  , 100.0  ,  0.0]
dt=0.25
x=[x0]
t0 = 0

Create a determinstic model from the paramters above.This will be used to create observations with a known beta (1.8 in the above)

In [ ]:
states = ['S','I','R']
extended_ode_deterministic= DeterministicOde(states, d, transition=transition_list)
extended_ode_deterministic.parameters =d
t=t0


In [ ]:
#create output with known beta value
t = numpy.linspace(t0, 100, 1000)
extended_ode_deterministic.initial_values = (x0, t[0])
# create a lsit of observations
Outputs = extended_ode_deterministic.integrate(t)

These outputs will be used to create "observations" for a second run which this time will have beta as a variable, and will have a Kalman Filter applied

In [ ]:
# now enter these outputs as observations
observations= [Output[1] for Output in Outputs]
observations =observations[2:]
t =t[:len(observations)]
observed_states =[1]

#observations = observations.tolist()
# set everything up again ,but this time with beta as a variable
transition_list = [
                  Transition(origin='S', destination='I', equation='beta*S*I/N', transition_type="T"),
                  Transition(origin='I', destination='R', equation='gamma*I', transition_type="T")
                 ]

d = dict()

d['gamma'] = 0.2
d['N']     = 1000

t1 = 100
N = 1000
# beta is the 4th item in the state list. The line below implies that beta is starting from
# a value of 10.0 . The true value used to create the input data is 1.8
x0=  [ 900.0  , 100.0  ,  0.0,10]
dt=0.25
x=[x0]
t0 = 0

Now run model with parameters above with Kalamn Filter integration

In [ ]:
states = ['S','I','R','beta']
extended_ode_deterministic_beta= DeterministicOde(states, d, transition=transition_list)
extended_ode_deterministic_beta.parameters =d
#indices of states whcih are being observed
observed_states =[1]
sigmap2 =10.0
sigmaw2 =1.0
sigmar2 =1.0
# Integrate the equation again ,this time with the Kalman Filter applied 
# The additional input arguments for the IntegrateWithKalamnFilter routine are:
# list of observations
# Indices of observed states
# The three Noise variances
# Index of the beta value in the list of variables
#
# The 3 variances are
# 'p' the starting value for the posteriori estimate covariance matrix 
# 'w' the process noise
# 'r' the observation noise
Outputs_beta =extended_ode_deterministic_beta.IntegrateWithKalmanFilter(extended_ode_deterministic_beta,x0,t, 

As beta is the last of the variables, extract the last item from the output of the Kalman filtered run and plot it. The true value used ot create the observations is 1.8, and the KF run started from an assumed vale of 10.0 .So thr graph should start higher, but should eventually converge to 1.8

In [ ]:
import matplotlib.pyplot as plt
## Extract the estimated beta values, iteration by iteration, from the most recent run output
beta_values=[]

for i in range(len(Outputs_beta)):
    beta_values.append(Outputs_beta[i][-1])
# Plot beta value - estimates are expected to converge on value of 1.8 as that was the true value used to 
# generate the input observations
plt.plot(beta_values)

plt.ylabel("Estimated Beta")
plt.xlabel("Iterations")
plt.title("Estimation of Beta Using Kalman Filter")
plt.show(block=False)

# References
[1] The Use of Kalman Filters to Detect Changing Infectivity Rates During a Pandemic
,David Mustard (Senior Data Scientist, UK Health Security Agency), Thomas Finnie (Principal Data Scientist, UK Health Security Agency) 

